# Mentoria Data Science Academy

## Imports

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, cohen_kappa_score, classification_report, confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

# Downloads do pacote NLTK
nltk.download('stopwords')
nltk.download('rslp')

## 0.0 Carregando os Dados

In [ ]:
data_path = 'dados'
df1 = pd.read_csv(os.path.join(data_path, 'posts.csv'), sep = '|')
df2 = pd.read_csv(os.path.join(data_path, 'posts_2.csv'), sep = '|')
df3 = pd.read_csv(os.path.join(data_path, 'posts_3.csv'), sep = '|')

In [ ]:
df1.head()

In [ ]:
df = pd.concat([df1, df2, df3], axis = 0)

In [ ]:
# As colunas visits e likes não estão presentes em todos os dfs, então vamos remove-las 
df.drop(columns=['visits', 'likes'], inplace=True)

In [ ]:
df.head()

In [ ]:
df['area'] = df['area'].apply(lambda x: x.split('.')[0])

## 1.0 Descrição dos dados

### 1.1 Dimensão dos dados

In [ ]:
print("Número de exemplos:", df.shape[0])
print("Número de colunas:", df.shape[1])

### 1.2 Tipo das colunas

In [ ]:
# Mostra o tipo de cada variável
df.dtypes

Precisamos converter a variável date para formato de data para então podermos utilizar informações como dia da semana, fim de semana, ou há quantos meses/anos foi feita a pergunta.

In [ ]:
# Espaço reservado para trabalhar na variável date

### 1.3 Variável target 

### 1.3.1 Classes únicas


In [ ]:
# Check de quais classes o dataset possui
df['area'].unique()

In [ ]:
print("Número de classes diferentes: ", len(df['area'].unique()))

In [ ]:
print('Número de observações por classe:')
df.groupby('area')['area'].count()

## Classes

Algumas classes estão com textos diferentes porém indicam a mesma coisa (Ex. 'direito-do-trabalho' e 'direito-do-trabalho/'). Iremos padronizar os textos para seguir com os seguintes labels:

- Administrativo
- Civil, Empresarial e do Consumidor
- Familia
- Trabalho
- Penal

In [ ]:
df['area'] = [
    'Administrativo' if any(x in area for x in ['administrativo']) else
    'Civil, Empresarial e do Consumidor' if any(x in area for x in ['civil', 'empresarial', 'consumidor']) else
    'Familia' if any(x in area for x in ['familia']) else
    'Trabalho' if any(x in area for x in ['trabalho']) else
    'Penal' if any(x in area for x in ['penal']) else
    area
    for area in df['area']
]

In [ ]:
df.groupby('area')['area'].count()

In [ ]:
print("Número de classes diferentes: ", len(df['area'].unique()))

### 1.3.2 Verificando o balanceamento das classes

In [ ]:
# Verifica em porcentagem o balanço das classes
df['area'].value_counts(normalize=True)*100

 ## 2.0 Limpeza dos dados

### 2.1 Visualizando os dados em formato RAW

In [ ]:
# Printando as 10 primeiras strings em formato raw
for i, post in enumerate(df['post'][:10]):
    print(repr(df['post'].iloc[i]))
    print('\n')

## 2.2 Criando um Pipeline de Limpeza

In [ ]:
# Essa função substitui mais de um espaço em sequência (\n\t\t\t\t) por apenas um espaço
def remove_seq_breaklines(text):
    clean_text = re.sub('\s{2,}', ' ', text)
    return clean_text

In [ ]:
# Essa função substitui \n por um espaço
def remove_breaklines(text):
    clean_text = re.sub('\n', ' ', text)
    return clean_text

In [ ]:
def remove_seq_spaces(text):
    clean_text = re.sub('[ ]{2,}', '', text)
    return clean_text

In [ ]:
remove_seq_spaces('eu  eu')

In [ ]:
# Essa função remove os espaços que existem no fim e no inicio
def remove_spaces(text):
    clean_text = text.strip(' ')
    clean_text
    return clean_text

In [ ]:
# Essa função remove as pontuações e os números (ex: datas serão removidas)
def remove_punctuation(text):
    clean_text  = "".join([char for char in text if char not in string.punctuation])
    clean_text = re.sub('[0-9]+', '', clean_text)
    clean_text = clean_text.replace('º', '')
    return clean_text

In [ ]:
# Essa função converte textos em textos lowercase 
def convert_to_lower(text):
    return text.lower()

In [ ]:
# Essa função remove as stopwords
def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    clean_text = [word for word in text.split(' ') if word not in stopwords]
   
    return " ".join(clean_text)


In [ ]:
# Essa função aplica o stemming nas palavras
def stemmatization(text):
    stemmer = nltk.stem.RSLPStemmer()
    split_text = text.split(' ')
    stem_text = [stemmer.stem(word) for word in split_text if word != ""]
    stem_text = " ".join(stem_text)
    
    return stem_text

In [ ]:
# Função que aplica o pipeline
def run_clean_pipeline(text):
    clean_text = remove_breaklines(text)
    clean_text = remove_seq_breaklines(clean_text)
    clean_text = remove_spaces(clean_text)
    clean_text = remove_punctuation(clean_text)
    clean_text = convert_to_lower(clean_text)
    clean_text = remove_stopwords(clean_text)
    clean_text = stemmatization(clean_text)

    return clean_text
    

In [ ]:
# Fazendo a limpeza das linhas através do pipeline
clean_posts = list(df['post'].apply(run_clean_pipeline))

In [ ]:
# Printa os 10 primeiras posts após a limpeza
for post in clean_posts[:10]:
    print(post)
    print('\n')

In [ ]:
df['clean_posts'] = clean_posts

## 3.0 Preparação dos Dados

### 3.1 Codificação por Label

In [ ]:
# Convertendo variáveis target para valores numéricos

le = LabelEncoder()
le.fit(df['area'])
df['area_encoded'] = le.transform(df['area'])

### 3.2 Split train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['area','area_encoded'], axis = 1), df['area_encoded'], test_size=0.33, shuffle = True, random_state = 42)


### 3.3 Transformação TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

train_tfidf = tfidf_vectorizer.fit_transform((X_train['clean_posts'])) 
test_tfidf = tfidf_vectorizer.transform((X_test['clean_posts']))

## 4.0 Modelo de Machine Learning

In [ ]:
model = MultinomialNB()
model.fit(train_tfidf, y_train)

In [ ]:
ypred_nb = model.predict(test_tfidf)

## 5.0 Avaliação dos Modelos

Optei por utilizar Accuracy, Precision e Kappa Score por ser um problema de classificação multi-classe.

- Accuracy mede o quão bem o modelo está fazendo as previsões.
- Precision mede o quão bem o modelo está fazendo as predições dos True Positives
- Kappa score é uma medida de concordância entre 2 estimadores, no caso, o modelo e o y_test.

### 5.1 Avaliação Individual do Modelo

In [ ]:
print('Accuracy:', accuracy_score(y_test, ypred_nb))
print('Precision:', precision_score(y_test, ypred_nb, average = 'macro' ))
print('Kappa Score:', cohen_kappa_score(y_test, ypred_nb))

In [ ]:
print(classification_report(y_test, ypred_nb))

In [ ]:
sns.heatmap(confusion_matrix(y_test, ypred_nb),fmt = 'd', annot = True)
plt.xlabel('Predictions')
plt.ylabel('True Labels')

### 5.2 Cross-Validation

In [ ]:
num_folds = 5
kfold = StratifiedKFold(n_splits = num_folds, shuffle = True, random_state = 42)

accuracy_list = []
precision_list = []
kappa_list = []

i = 1

for train_ix, val_ix in kfold.split(df.drop('area', axis = 1), df['area']):
    print('Fold: {} / {}'.format(i, num_folds))
    
    X_train_fold = df.iloc[train_ix]
    y_train_fold = df['area'].iloc[train_ix]
    
    X_val_fold = df.iloc[val_ix]
    y_val_fold = df['area'].iloc[val_ix]
    
    tfidf_vectorizer = TfidfVectorizer()

    train_tfidf_fold = tfidf_vectorizer.fit_transform((X_train_fold['clean_posts'])) 
    val_tfidf_fold = tfidf_vectorizer.transform((X_val_fold['clean_posts']))
    
    model = MultinomialNB()
    model.fit(train_tfidf_fold, y_train_fold)
    
    y_pred_cv = model.predict(val_tfidf_fold)
    
    #metrics
    
    #Accuracy
    acc = accuracy_score(y_val_fold, y_pred_cv)
    accuracy_list.append(acc)
    
    #Precision
    prec = precision_score(y_val_fold, y_pred_cv, average = 'macro')
    precision_list.append(prec)
    
    #Kappa
    kappa = cohen_kappa_score(y_val_fold, y_pred_cv)
    kappa_list.append(kappa)
    
    
    i+=1
    
    

In [ ]:
print('Cross-Validation Evaluation:')
print('Accuracy: {:.2f} +/- {:.2f}'.format(np.mean(accuracy_list),np.std(accuracy_list)))
print('Precision: {:.2f} +/- {:.2f}'.format(np.mean(precision_list),np.std(precision_list)))
print('Kappa Score: {:.2f} +/- {:.2f}'.format(np.mean(kappa_list),np.std(kappa_list)))

## Sugestões de Melhorias:

- Utilização de Bigramas ou Trigramas antes da transformação TF-IDF com o próprio MultinomialNB
- Utilização de SVM c/ Linear Kernel
- Utilização de Redes Neurais - LSTM
- SMOTE para um melhor balanceamento das classes
- Análise da Nuvem de palavras pra cada classe, a fim de identificar quais termos aparecem com mais frequência dependendo da classe.